In [16]:
from pyspark.sql import SparkSession
from datetime import datetime


In [17]:
warehouse_location = '/user/carpetas_inv'

In [ ]:
!hdfs dfs -ls /user
!hdfs dfs -mkdir /user/carpetas_inv
!wget https://storage.googleapis.com/public-bucket-up-2022/carpetas_investigacion.csv
!hdfs dfs -moveFromLocal carpetas_investigacion.csv /user/carpetas_inv/

In [2]:
#en hive, crear tabla como externa.
"""
hive>
CREATE EXTERNAL TABLE carpetas_investigacion
(
  id INT ,
  ao_hechos INT,
  mes_hechos STRING,
  fecha_hechos TIMESTAMP,
  delito STRING,
  categoria_delito STRING,
  fiscalia STRING ,
  agencia STRING ,
  unidad_investigacion STRING,
  colonia_hechos STRING ,
  alcaldia_hechos STRING,
  fecha_inicio TIMESTAMP ,
  mes_inicio STRING,
  ao_inicio INT,
  calle_hechos STRING ,
  calle_hechos2 STRING,
  longitud float,
  latitud float,
  geopoint STRING  
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
location '/user/carpetas_inv';
"""

"\nhive>\nCREATE EXTERNAL TABLE carpetas_investigacion\n(\n  id INT ,\n  ao_hechos INT,\n  mes_hechos STRING,\n  fecha_hechos TIMESTAMP,\n  delito STRING,\n  categoria_delito STRING,\n  fiscalia STRING ,\n  agencia STRING ,\n  unidad_investigacion STRING,\n  colonia_hechos STRING ,\n  alcaldia_hechos STRING,\n  fecha_inicio TIMESTAMP ,\n  mes_inicio STRING,\n  ao_inicio INT,\n  calle_hechos STRING ,\n  calle_hechos2 STRING,\n  longitud float,\n  latitud float,\n  geopoint STRING  \n)\nROW FORMAT DELIMITED\nFIELDS TERMINATED BY ','\nLINES TERMINATED BY '\n'\nSTORED AS TEXTFILE\nlocation '/user/carpetas_inv';\n"

In [18]:
spark = SparkSession \
    .builder \
    .appName("Mi primer app de Spark") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

22/04/18 18:43:03 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/18 18:43:03 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/18 18:43:03 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/18 18:43:03 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [19]:
spark.sql("SELECT * FROM carpetas_investigacion limit 10").show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


+----+---------+----------+-------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+-------------------+-------------------+----------+---------+--------------------+--------------------+----------+---------+--------------+
|  id|ao_hechos|mes_hechos|       fecha_hechos|              delito|    categoria_delito|            fiscalia|agencia|unidad_investigacion|      colonia_hechos|    alcaldia_hechos|       fecha_inicio|mes_inicio|ao_inicio|        calle_hechos|       calle_hechos2|  longitud|  latitud|      geopoint|
+----+---------+----------+-------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+-------------------+-------------------+----------+---------+--------------------+--------------------+----------+---------+--------------+
|null|     null|mes_hechos|               null|              delito|    categoria_delito|           

In [20]:
ubicaciones = spark.sql("SELECT delito, fecha_hechos, fiscalia, latitud, longitud FROM carpetas_investigacion where alcaldia_hechos = 'TLALPAN' and latitud is not null")

In [21]:
ubicaciones

DataFrame[delito: string, fecha_hechos: timestamp, fiscalia: string, latitud: float, longitud: float]

In [22]:
ubicaciones.show()

+--------------------+-------------------+--------------------+---------+----------+
|              delito|       fecha_hechos|            fiscalia|  latitud|  longitud|
+--------------------+-------------------+--------------------+---------+----------+
|ROBO A NEGOCIO SI...|2016-01-05 15:10:00|INVESTIGACIÓN EN ...|19.289762| -99.17881|
|ROBO DE ACCESORIO...|2018-05-22 20:30:00|INVESTIGACIÓN EN ...|19.281824| -99.17947|
|ROBO DE VEHICULO ...|2018-05-14 12:00:00|INVESTIGACIÓN EN ...|19.288815| -99.22349|
|FALSIFICACION DE ...|2018-05-19 11:00:00|INVESTIGACIÓN EN ...| 19.29802| -99.13836|
|     ROBO DE OBJETOS|2016-01-15 07:20:00|INVESTIGACIÓN EN ...|19.285206|-99.218414|
|ROBO A CASA HABIT...|2015-01-14 15:00:00|INVESTIGACIÓN EN ...|19.264786|-99.207695|
|ROBO DE OBJETOS D...|2018-05-27 10:30:00|INVESTIGACIÓN EN ...|19.291553| -99.16226|
|  VIOLENCIA FAMILIAR|2018-05-27 03:00:00|INVESTIGACIÓN EN ...|19.267105| -99.23836|
|   LESIONES CULPOSAS|2015-06-18 00:00:00|INVESTIGACIÓN EN ...|19

In [23]:
ubicaciones.printSchema()

root
 |-- delito: string (nullable = true)
 |-- fecha_hechos: timestamp (nullable = true)
 |-- fiscalia: string (nullable = true)
 |-- latitud: float (nullable = true)
 |-- longitud: float (nullable = true)



In [24]:
ubic_pandas = ubicaciones.toPandas()

In [25]:
ubic_pandas

,delito,fecha_hechos,fiscalia,latitud,longitud
0,ROBO A NEGOCIO SIN VIOLENCIA,2016-01-05 15:10:00,INVESTIGACIÓN EN TLALPAN,19.289762,-99.178810
1,ROBO DE ACCESORIOS DE AUTO,2018-05-22 20:30:00,INVESTIGACIÓN EN TLALPAN,19.281824,-99.179466
2,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,2018-05-14 12:00:00,INVESTIGACIÓN EN TLALPAN,19.288815,-99.223488
3,FALSIFICACION DE TITULOS AL PORTADOR Y DOCUMEN...,2018-05-19 11:00:00,INVESTIGACIÓN EN TLALPAN,19.298019,-99.138359
4,ROBO DE OBJETOS,2016-01-15 07:20:00,INVESTIGACIÓN EN TLALPAN,19.285206,-99.218414
...,...,...,...,...,...
43507,QUEBRANTAMIENTO DE SELLOS,2018-04-03 09:00:00,INVESTIGACIÓN EN DELITOS AMBIENTALES Y EN MATE...,19.285286,-99.128784
43508,ROBO DE ACCESORIOS DE AUTO,2017-09-21 07:00:00,INVESTIGACIÓN EN TLALPAN,19.210995,-99.200905
43509,ROBO A PASAJERO / CONDUCTOR DE TAXI CON VIOLENCIA,2018-07-09 20:00:00,INVESTIGACIÓN EN COYOACÁN,19.221430,-99.210609
43510,AMENAZAS,2018-07-26 12:00:00,INVESTIGACIÓN EN TLALPAN,19.214911,-99.177666


In [30]:
ubic_pandas = ubic_pandas[ubic_pandas["fecha_hechos"].apply(lambda x: x.year == 2018)]

In [31]:
import folium as fl
import pandas as pd
import numpy as np

In [34]:
m = fl.Map(
    location = [19.425665, -99.154586],
    zoom_start = 14,
)

In [35]:
counter = 0
for i in ubic_pandas.index:    
    if ~np.isnan(ubic_pandas.latitud[i]) and ~np.isnan(ubic_pandas.longitud[i]):
        #print(ubic_pandas.loc[i, "longitud"], ~np.isnan(ubic_pandas.longitud[i]))
        fl.Marker([ubic_pandas.loc[i, "latitud"], ubic_pandas.loc[i, "longitud"] ] ).add_to(m)
    counter += 1
    if counter > 100:
        break

In [36]:
m

In [37]:
#Getting flashy
ubic_pandas.shape

(13486, 5)

In [38]:
m1 = fl.Map(
    location = [19.425665, -99.154586],
    zoom_start = 14,
)

counter = 0
for i in ubic_pandas.index:
    if ~np.isnan(ubic_pandas.latitud[i]) and ~np.isnan(ubic_pandas.longitud[i]):  
        if ubic_pandas.loc[i, "fiscalia"].strip() != "INVESTIGACIÓN EN TLALPAN":
            fl.Marker(
                location = [ubic_pandas.loc[i, "latitud"], ubic_pandas.loc[i, "longitud"] ],
                popup = ubic_pandas.loc[i, "delito"], clustered_marker = True        
            ).add_to(m1)
        else:
            fl.Marker(
                location = [ubic_pandas.loc[i, "latitud"], ubic_pandas.loc[i, "longitud"] ],
                popup = ubic_pandas.loc[i, "delito"],
                icon = fl.Icon(color="red", icon="info-sign"), clustered_marker = True
            ).add_to(m1)
    counter += 1
    if counter > 100:
        break

In [39]:
m1

In [ ]:
#otros ejemplos
#https://python-visualization.github.io/folium/quickstart.html

In [40]:
spark.stop()